<details><summary> </summary>

# Skip notebook test

</details>

In [ ]:
import pandas as pd
import numpy as np
import requests
from scipy.spatial import distance
import notebook_utils.notebook_helpers as utils

# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

While other notebooks such as [cvrptw_service_team_routing.ipynb](cvrptw_service_team_routing.ipynb) focus on the cuOpt API and high level problem modeling, here we focus on performance.

cuOpt offers a unique benefit over other solver_settingss, specifically, time to solution.  In addition to achieving world class accuracy, cuOpt also produces these solutions in a time frame that allows for re-optimization in dynamic environments and rapid iteration over possible problem configurations.

Here we are demonstrating this performance on a large popular academic [dataset by Gehing & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/).  These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems.  Each problem instance has an associated best known solution, the one we will measure against is shown below

**API Reference**: [cuOpt Server Documentation](https://docs.nvidia.com/cuopt/serv_api.html)

In [ ]:
homberger_1000_file = 'notebook_utils/data/C1_10_1.TXT'

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Problem Data
The data for this problem instance are provided via text file.  cuOpt has a utility function available specifically for the Gehring & Homberger benchmark which converts the problem into the components required by cuOpt.

In [ ]:
orders, vehicle_capacity, n_vehicles = utils.create_from_file(homberger_1000_file)
n_locations = orders["demand"].shape[0]-1
print("Number of locations          : ", n_locations)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

### Setup the cuOpt server and test the health of the server

**NOTE**: Please update **ip** and **port** on which the server is running.

In [ ]:
ip = "0.0.0.0"
port = "5000"
url = "http://" + ip + ":" + port + "/cuopt/"

# Test the health of the cuOpt server
assert requests.get(url + "health").status_code == 200

### Cost Matrix

In [ ]:
coords = list(zip(orders['xcord'].to_list(),
                  orders['ycord'].to_list()))

cost_matrix = pd.DataFrame(distance.cdist(coords, coords, 'euclidean')).astype(np.float32)
print(f"Shape of cost matrix: {cost_matrix.shape}")

### Set Cost Matrix

In [ ]:
data_params = {"return_data_state": False}
cost_data = {"cost_matrix": {0: cost_matrix.values.tolist()}}
response_set = requests.post(
    url + "add_cost_matrix", params=data_params, json=cost_data
)
assert response_set.status_code == 200

### Set Fleet Data

In [ ]:
# Set the fleet data
vehicle_locations = [[0, 0]] * n_vehicles
fleet_data = {
    "vehicle_locations": vehicle_locations,
    "capacities": [[vehicle_capacity] * n_vehicles],
}

# Dispatch the fleet data to the cuOpt server
response_set = requests.post(
    url + "set_fleet_data", json=fleet_data
)
assert response_set.status_code == 200

### Set Task Data

In [ ]:
# Set the task data
task_data = {
    "task_locations": orders['vertex'].values.tolist(),
    "task_time_windows": list(zip(orders['earliest_time'].values.tolist(),
                  orders['latest_time'].values.tolist())),
    "service_times": orders['service_time'].values.tolist(),
    "demand": [orders['demand'].values.tolist()],
}

# Dispatch the task data to the cuOpt server
response_set = requests.post(
    url + "set_task_data", json=task_data
)
assert response_set.status_code == 200

### Set Solver configuration

In [ ]:
solver_settings = {
    "time_limit": 0.5,
    "number_of_climbers": 2048,
}
# set number of climbers that will try to search for an optimal routes in parallel
response_set = requests.post(
    url + "set_solver_config", json=solver_settings
)
assert response_set.status_code == 200

### Helper functions to solve and process the output

In [ ]:
# Here we will examine the quality of the solution we increase the time budget provided to cuOpt
def solve_problem(problem_size):
    solver_response = requests.get(url + "get_optimized_routes")
    solver_resp = solver_response.json()["response"]["solver_response"]
    if solver_resp["status"] == 0:
        print("Cost for the routing in time: ", solver_resp["solution_cost"])
        print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
        utils.show_vehicle_routes(solver_resp, ["Depot"]+[str(i) for i in range(1, problem_size+1)])
    else:
        print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])
        
    return(solver_resp["num_vehicles"], solver_resp["solution_cost"])

def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

### Get Optimized Results

Update solver config and test different run-time 

**1 Second Time Limit**

In [ ]:
solver_settings["time_limit"] = 1
# update the time limit for solving the problem
response_set = requests.put(
    url + "update_solver_config", json=solver_settings
)
assert response_set.status_code == 200
# re-solve the problem with time limit equals 1
vehicles, cost = solve_problem(len(cost_matrix))

In [ ]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

**10 Second Time Limit**

In [ ]:
solver_settings["time_limit"] = 10
# update the time limit for solving the problem
response_set = requests.put(
    url + "update_solver_config", json=solver_settings
)
assert response_set.status_code == 200
# re-solve the problem with time limit equals ten
vehicles, cost = solve_problem(len(cost_matrix))

In [ ]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

**20 Second Time Limit**

In [ ]:
solver_settings["time_limit"] = 20
# update the time limit for solving the problem
response_set = requests.put(
    url + "update_solver_config", json=solver_settings
)
assert response_set.status_code == 200
# re-solve the problem with time limit equals twenty
vehicles, cost = solve_problem(len(cost_matrix))

In [ ]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

_____

#### SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

#### SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the "Software"),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the
Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
DEALINGS IN THE SOFTWARE.

---